In [468]:
%load_ext blackcellmagic

from isolation import Isolation, DebugState
from my_custom_player import MAX_BOOK_LEVEL
import itertools
import random

WIDTH = 11
PADDED_WIDTH = WIDTH + 2 # Adjusting for our soft bitmap border
HEIGHT = 9

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


SyntaxError: invalid syntax (my_custom_player.py, line 128)

In [113]:
# Will probabalistically sample random depth-searches, finding average utility at each.
# We are collecting possible positions in the first three levels of the game after a first choice is made.

BOOK = {}

Now it's possible to sample only from a limited starting set, and take advantage of symmetry to infer what the combinations would be given a different starting position.

We will use the bottom-right portion of the board as the canoncial "starting quadrant".

![alt text](img/911symmetries.png)


In [357]:
def xy2ind(x, y):
    """
    Return a bitboard index for an x,y pair, where the (0,0) coordinate is bottom-right
    and values increase to the left and upward.
    """
    return PADDED_WIDTH * y + x


# Calculate the minimum set of positions we need to capture data
# that is generalizable via symmetry
starting_set = [xy2ind(x, y) for x, y in itertools.product(range(6), range(5))]

# Calculate the second set of positions possible given the starting set of positions
def get_next_positions(action):
    game = Isolation()
    game = game.result(action)
    game.player = lambda: 0

    next_positions = []
    for action in game.actions():
        res = game.result(action)
        next_positions.append(res.locs[0])

    return next_positions




In [157]:
def expand_path(path):
    """
    Expand a path by returning the next set of viable moves,
    where 'path' is a tuple of p1 and p2 positions: (p1_position, p2_position, p1_position, ...)
    
    Returns a list of expanded paths.
    """
    assert (
        len(path) >= 2
    ), "Can only expand paths with where there are at least 2 arbitrary moves chosen"
    next_positions = get_next_positions(
        path[-2]
    )  # next move depends on what move was tried before, which was 2-back

    return [
        tuple(list(path) + [next_position])
        for next_position in next_positions
        if next_position not in path # ensure we never occupy a discarded square
    ]


def get_canonical_book(starting_set, level=3):
    assert level % 2 == 1, "Only odd levels are useful for player 1!"
    assert level >= 3, "We must create a book with more than 3 levels to be useful."

    p2_starting_set = (
        Isolation().actions()
    )  # player 2 can take any free position (that hasn't been occupied)
    book = {}

    element = {"wins": 0, "losses": 0}

    # Seed book with starting sets.
    # The book's keys are action paths through: (p1_position, p2_position, p1_position, ...) etc.
    book.update({(x,): element.copy() for x in starting_set})
    book.update(
        {
            (x, y): element.copy()
            for x, y in itertools.product(starting_set, p2_starting_set)
            if x != y
        }
    )  # level 2

    for level in range(3, level + 1):
        last_level_keys = [k for k in book.keys() if len(k) == level - 1]

        # For each key, expand it with the next set of possible moves.
        key_expansions = [expand_path(path) for path in last_level_keys]
        key_expansions = [path for paths in key_expansions for path in paths]  # flatten

        book.update({path: element.copy() for path in key_expansions})

    return book

In [163]:
BOOK = get_canonical_book(starting_set, MAX_BOOK_LEVEL)

In [164]:
print(f"Book contains {len(BOOK)} keys.")

Book contains 644292 keys.


In [165]:
from multiprocessing import Pool

# Simulate and count wins vs losses

def simulate(sims):
    book = BOOK.copy()
    for sim in range(sims):
        game = Isolation()

        action = random.choice(starting_set)
        positions = [action] # the first 'action' is a position on the board
        
        game = game.result(action)

        # Random choices until terminal state
        while not game.terminal_test():
            action = random.choice(game.actions())
            active_player = game.player()
            game = game.result(action)
            
            if len(positions) < MAX_BOOK_LEVEL:
                positions.append(game.locs[active_player])

        win = game.utility(0) == float("inf")

        for terminal_idx in range(1, MAX_BOOK_LEVEL + 1):
            key = tuple(positions[0:terminal_idx])
            if win:
                book[key]["wins"] += 1
            else:
                book[key]["losses"] += 1
    
    return book

In [198]:
with Pool(processes=7) as pool:
    %time books = pool.map(simulate, [int(1e7)]*7)

CPU times: user 23.7 s, sys: 9.81 s, total: 33.5 s
Wall time: 13h 10min 10s


In [ ]:
# Reduce step
from functools import reduce

def reduce_book(main_book, marginal_book):
    for k,v in marginal_book.items():
        main_book[k]["wins"] += v["wins"]
        main_book[k]["losses"] += v["losses"]
    
    return main_book

%time resulting_book = reduce(reduce_book, [BOOK, resulting_book] + books)

CPU times: user 4.23 s, sys: 568 ms, total: 4.8 s
Wall time: 5.12 s


In [ ]:
import pickle
import os

def save_book(version):
    filename = f"data/book_{version}.pkl"
    assert not os.path.exists(filename), "Don't overwrite your file!"
    with open(filename, 'wb') as f:
        pickle.dump(resulting_book, f)
        
save_book(10)

## Calculate win/loss ratios for each path

And for all transformations of paths, to have full coverage of all paths.

In [450]:
with open('data/book_10.pkl', 'rb') as f:
    full_book = pickle.load(f)

In [452]:
full_book = {k: v["wins"] / (v["wins"] + v["losses"] + 1e-6) for k,v in full_book.items()}

In [521]:
def save_ratio_book(book, version):
    filename = f"data.pickle"
    assert not os.path.exists(filename), "Don't overwrite your file!"
    with open(filename, 'wb') as f:
        pickle.dump(book, f)
        
save_ratio_book(full_book, 0)

In [495]:
current = Isolation().board ^ (1 << 99)

In [496]:
import math

diff = Isolation().board - current

res = int(math.log(diff) / math.log(2)) if diff != 0 else None

In [507]:
DebugState.ind2xy(71)

(6, 5)

In [508]:
from isolation.isolation import Action

In [510]:
Action.ESE

<Action.ESE: -15>

In [526]:
board = reduce(lambda x,y: x ^ (1 << y), [Isolation().board] + list((28,)))

In [527]:
print(DebugState.from_state(Isolation(board=board)))


+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   | X |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +



In [532]:
resulting_book[(28, 129,)]

KeyError: (28, 129)

In [536]:
BOOK[(28,129,)]

KeyError: (28, 129)

In [539]:
Isolation().actions()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114]